In [1]:
import pandas as pd
import glob
import math
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

Create 2011 gentrification prediction dataset:

1. Get list of MSOAs for each city as base table
2. Merge census variables to base table
3. Merge additional tables to base table
4. On a copy of the base table from 1, label gentrification using 2011 & 2021 data (nevermind, doing PCA)

Freeman gentrification labeling method:

a census tract is (1) marked eligible if (a) housing construction is below a metropolitan median, and (b) income is below the median, and (c) the tract is located in a central city. It is then (2) marked as gentrified if, after the measurement period, (a) there is a greater increase in educational attainment compared to the median and (b) an increase in real housing prices


Data sources:

2011 construction:
no direct numbers at the lsoa or msoa level, only regional & country stats
can use c2011ks401ew & c2011ks401uk, which contains number of household dwellings, subtract 2001 numbers, to get new dwellings as a proxy

2011 income:
.\data\raw\govuk\1smallareaincomeestimatesdatatcm77420299.xls

2011 cities:
.\data\raw\geoportal\Middle_Layer_Super_Output_Area_(2011)_to_Major_Towns_and_Cities_(December_2015)_Lookup_in_England_and_Wales.csv

2011 educational attainment:
c2011ks501ew

2021 educational attainment:
c2021ts067

2011 house prices:
data\raw\ons\hpssamedianpricebymsoa.xlsx

2021 house prices:
data\raw\ons\hpssamedianpricebymsoa.xlsx

In [2]:
base2011_df = pd.read_csv(r'.\data\raw\geoportal\Middle_Layer_Super_Output_Area_(2011)_to_Major_Towns_and_Cities_(December_2015)_Lookup_in_England_and_Wales.csv')
base2011_df = base2011_df.dropna(axis=0, how='any', ignore_index=True)
base2011_df = base2011_df.drop(columns='FID')
base2011_df

,MSOA11CD,MSOA11NM,TCITY15CD,TCITY15NM
0,E02000053,Barnet 030,J01000055,London
1,E02000209,Croydon 016,J01000055,London
2,E02000054,Barnet 031,J01000055,London
3,E02000055,Barnet 032,J01000055,London
4,E02000056,Barnet 033,J01000055,London
...,...,...,...,...
3255,E02003999,Carlisle 013,J01000021,Carlisle
3256,E02005068,Maidstone 001,J01000022,Chatham
3257,E02004497,Chelmsford 013,J01000023,Chelmsford
3258,E02004493,Chelmsford 009,J01000023,Chelmsford


In [3]:
metadata_2011 = pd.read_csv(r'.\data\raw\api\msoa\c2011\metadata.csv')
metadata_2011['clean_mnemonic'] = metadata_2011['mnemonic'].str.extract('(c2011ks\d*).*')
metadata_2011_mnemonics = list(set(metadata_2011['clean_mnemonic']))
len(metadata_2011_mnemonics)

35

In [4]:
c2011_dfs = []

for mnemonic in metadata_2011_mnemonics:

    root_dir = r'.\data\raw\api\msoa\c2011'
    mnemonic_tables = glob.glob(mnemonic+'*', root_dir=root_dir)
    mnemonic_dfs = []

    for table in mnemonic_tables:
        df = pd.read_csv(rf"{root_dir}\{table}")
        mnemonic_dfs.append(df)

    mnemonic_df = pd.concat(mnemonic_dfs)

    mnemonic_df = mnemonic_df.drop(columns=['Unnamed: 0', 'GEOGRAPHY_TYPE', 'GEOGRAPHY_TYPECODE', 'DATE'])
    mnemonic_df['OBS_VALUE'] = mnemonic_df['OBS_VALUE'].astype(float)
    mnemonic_df = mnemonic_df[mnemonic_df['OBS_VALUE'] > 0]
    cols = list(mnemonic_df.columns)
    var_col = [col for col in cols if col not in ['GEOGRAPHY_CODE', 'OBS_VALUE']][0]
    mnemonic_df = mnemonic_df.pivot_table(index='GEOGRAPHY_CODE', columns=var_col, values='OBS_VALUE', aggfunc='mean', fill_value=0)
    mnemonic_df = mnemonic_df.reset_index(drop=False)
    c2011_dfs.append(mnemonic_df)
c2011_dfs[0]

CELL_NAME,GEOGRAPHY_CODE,All categories: Highest level of qualification,Full-time students: Age 18 to 74: Economically active: In employment,Full-time students: Age 18 to 74: Economically active: Unemployed,Full-time students: Age 18 to 74: Economically inactive,Highest level of qualification: Apprenticeship,Highest level of qualification: Level 1 qualifications,Highest level of qualification: Level 2 qualifications,Highest level of qualification: Level 3 qualifications,Highest level of qualification: Level 4 qualifications and above,Highest level of qualification: Other qualifications,No qualifications,Schoolchildren and full-time students: Age 16 to 17,Schoolchildren and full-time students: Age 18 and over
0,E02000001,6755.0,103.0,30.0,289.0,47.0,291.0,445.0,484.0,4618.0,416.0,454.0,69.0,422.0
1,E02000002,5024.0,84.0,24.0,163.0,118.0,789.0,706.0,479.0,914.0,395.0,1623.0,163.0,272.0
2,E02000003,7798.0,161.0,44.0,235.0,169.0,1210.0,1236.0,847.0,1829.0,729.0,1778.0,226.0,442.0
3,E02000004,4986.0,74.0,25.0,113.0,163.0,800.0,825.0,539.0,891.0,266.0,1502.0,185.0,215.0
4,E02000005,6362.0,107.0,51.0,175.0,160.0,1026.0,1038.0,653.0,1119.0,527.0,1839.0,232.0,333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7196,W02000419,8748.0,106.0,24.0,151.0,372.0,1365.0,1536.0,1068.0,1526.0,387.0,2494.0,249.0,283.0
7197,W02000420,11092.0,126.0,26.0,141.0,468.0,1287.0,1709.0,1248.0,3921.0,398.0,2061.0,320.0,293.0
7198,W02000421,9693.0,111.0,24.0,190.0,319.0,1139.0,1465.0,1161.0,3150.0,360.0,2099.0,228.0,332.0
7199,W02000422,5579.0,181.0,24.0,231.0,64.0,386.0,525.0,659.0,3396.0,275.0,274.0,24.0,436.0


In [5]:
c2011_dfs[0][c2011_dfs[0]['GEOGRAPHY_CODE'] == 'E02000053']

CELL_NAME,GEOGRAPHY_CODE,All categories: Highest level of qualification,Full-time students: Age 18 to 74: Economically active: In employment,Full-time students: Age 18 to 74: Economically active: Unemployed,Full-time students: Age 18 to 74: Economically inactive,Highest level of qualification: Apprenticeship,Highest level of qualification: Level 1 qualifications,Highest level of qualification: Level 2 qualifications,Highest level of qualification: Level 3 qualifications,Highest level of qualification: Level 4 qualifications and above,Highest level of qualification: Other qualifications,No qualifications,Schoolchildren and full-time students: Age 16 to 17,Schoolchildren and full-time students: Age 18 and over
51,E02000053,7153.0,262.0,189.0,718.0,86.0,718.0,814.0,751.0,2473.0,1199.0,1112.0,165.0,1169.0


In [6]:
c2011_census_df = base2011_df.copy()
for df in c2011_dfs:
    c2011_census_df = c2011_census_df.merge(df, how='left', left_on='MSOA11CD', right_on='GEOGRAPHY_CODE', suffixes=('', '_y'))
    c2011_census_df_cols = list(c2011_census_df.columns)
    c2011_census_df_dup_cols = [col for col in c2011_census_df_cols if '_y' in col]
    c2011_census_df_dup_cols.append('GEOGRAPHY_CODE')
    c2011_census_df = c2011_census_df.drop(columns=c2011_census_df_dup_cols)
c2011_census_df = c2011_census_df.dropna(axis=1, how='all')
c2011_census_df

,MSOA11CD,MSOA11NM,TCITY15CD,TCITY15NM,All categories: Highest level of qualification,Full-time students: Age 18 to 74: Economically active: In employment,Full-time students: Age 18 to 74: Economically active: Unemployed,Full-time students: Age 18 to 74: Economically inactive,Highest level of qualification: Apprenticeship,Highest level of qualification: Level 1 qualifications,...,Medical and care establishment: Local Authority: Children's home (including secure units),Medical and care establishment: NHS: General hospital,Medical and care establishment: NHS: Mental health hospital/unit (including secure units),Medical and care establishment: NHS: Other hospital,Medical and care establishment: Other,Medical and care establishment: Other: Care home with nursing,Medical and care establishment: Other: Care home without nursing,Medical and care establishment: Other: Children's home (including secure units),Medical and care establishment: Registered Social Landlord/Housing Association,Other establishments
0,E02000053,Barnet 030,J01000055,London,7153.0,262.0,189.0,718.0,86.0,718.0,...,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,428.5
1,E02000209,Croydon 016,J01000055,London,6136.0,245.0,69.0,255.0,92.0,959.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2,E02000054,Barnet 031,J01000055,London,5907.0,175.0,57.0,446.0,64.0,442.0,...,0.0,0.0,0.0,0.0,5.0,0.0,26.0,0.0,0.0,0.0
3,E02000055,Barnet 032,J01000055,London,9137.0,345.0,146.0,929.0,79.0,677.0,...,0.0,0.0,0.0,0.0,0.0,0.0,106.0,0.0,0.0,130.5
4,E02000056,Barnet 033,J01000055,London,7192.0,83.0,17.0,282.0,45.0,402.0,...,0.0,0.0,0.0,0.0,0.0,86.0,81.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,E02003999,Carlisle 013,J01000021,Carlisle,7862.0,78.0,15.0,72.0,410.0,1145.0,...,0.0,0.0,12.0,0.0,0.0,33.0,33.0,0.0,0.0,103.0
3256,E02005068,Maidstone 001,J01000022,Chatham,5659.0,89.0,12.0,47.0,262.0,913.0,...,0.0,0.0,0.0,0.0,0.0,63.0,4.0,2.0,0.0,6.0
3257,E02004497,Chelmsford 013,J01000023,Chelmsford,6535.0,74.0,7.0,81.0,252.0,876.0,...,15.0,0.0,0.0,0.0,0.0,39.0,13.0,0.0,0.0,2.0
3258,E02004493,Chelmsford 009,J01000023,Chelmsford,8147.0,167.0,26.0,157.0,196.0,1107.0,...,0.0,0.0,0.0,0.0,0.0,128.0,0.0,0.0,0.0,15.5


In [7]:
income2011_df = pd.read_excel(r'.\data\raw\govuk\1smallareaincomeestimatesdatatcm77420299.xls', sheet_name='Total weekly income', header=4)
income2011_df = income2011_df[['MSOA code', 'Total weekly income (£)']]
income2011_df = income2011_df.dropna(axis=0, how='any')
income2011_df

c2011_income_df = c2011_census_df.copy()
c2011_income_df = c2011_income_df.merge(income2011_df, how='left', left_on='MSOA11CD', right_on='MSOA code')
c2011_income_df = c2011_income_df.drop(columns='MSOA code')
c2011_income_df

,MSOA11CD,MSOA11NM,TCITY15CD,TCITY15NM,All categories: Highest level of qualification,Full-time students: Age 18 to 74: Economically active: In employment,Full-time students: Age 18 to 74: Economically active: Unemployed,Full-time students: Age 18 to 74: Economically inactive,Highest level of qualification: Apprenticeship,Highest level of qualification: Level 1 qualifications,...,Medical and care establishment: NHS: General hospital,Medical and care establishment: NHS: Mental health hospital/unit (including secure units),Medical and care establishment: NHS: Other hospital,Medical and care establishment: Other,Medical and care establishment: Other: Care home with nursing,Medical and care establishment: Other: Care home without nursing,Medical and care establishment: Other: Children's home (including secure units),Medical and care establishment: Registered Social Landlord/Housing Association,Other establishments,Total weekly income (£)
0,E02000053,Barnet 030,J01000055,London,7153.0,262.0,189.0,718.0,86.0,718.0,...,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,428.5,810.0
1,E02000209,Croydon 016,J01000055,London,6136.0,245.0,69.0,255.0,92.0,959.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,810.0
2,E02000054,Barnet 031,J01000055,London,5907.0,175.0,57.0,446.0,64.0,442.0,...,0.0,0.0,0.0,5.0,0.0,26.0,0.0,0.0,0.0,950.0
3,E02000055,Barnet 032,J01000055,London,9137.0,345.0,146.0,929.0,79.0,677.0,...,0.0,0.0,0.0,0.0,0.0,106.0,0.0,0.0,130.5,870.0
4,E02000056,Barnet 033,J01000055,London,7192.0,83.0,17.0,282.0,45.0,402.0,...,0.0,0.0,0.0,0.0,86.0,81.0,0.0,0.0,3.0,1300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,E02003999,Carlisle 013,J01000021,Carlisle,7862.0,78.0,15.0,72.0,410.0,1145.0,...,0.0,12.0,0.0,0.0,33.0,33.0,0.0,0.0,103.0,690.0
3256,E02005068,Maidstone 001,J01000022,Chatham,5659.0,89.0,12.0,47.0,262.0,913.0,...,0.0,0.0,0.0,0.0,63.0,4.0,2.0,0.0,6.0,1020.0
3257,E02004497,Chelmsford 013,J01000023,Chelmsford,6535.0,74.0,7.0,81.0,252.0,876.0,...,0.0,0.0,0.0,0.0,39.0,13.0,0.0,0.0,2.0,960.0
3258,E02004493,Chelmsford 009,J01000023,Chelmsford,8147.0,167.0,26.0,157.0,196.0,1107.0,...,0.0,0.0,0.0,0.0,128.0,0.0,0.0,0.0,15.5,940.0


In [8]:
housing2011_df = pd.read_excel(r'.\data\raw\ons\hpssamedianpricebymsoa.xlsx', sheet_name='1a', header=2)
housing2011_df = housing2011_df[['MSOA code', 'Year ending Dec 2011']]
housing2011_df = housing2011_df.rename(columns={'Year ending Dec 2011': 'Median house price'})
housing2011_df

c2011_housing_df = c2011_income_df.copy()
c2011_housing_df = c2011_housing_df.merge(housing2011_df, how='left', left_on='MSOA11CD', right_on='MSOA code')
c2011_housing_df = c2011_housing_df.drop(columns='MSOA code')
c2011_housing_df

,MSOA11CD,MSOA11NM,TCITY15CD,TCITY15NM,All categories: Highest level of qualification,Full-time students: Age 18 to 74: Economically active: In employment,Full-time students: Age 18 to 74: Economically active: Unemployed,Full-time students: Age 18 to 74: Economically inactive,Highest level of qualification: Apprenticeship,Highest level of qualification: Level 1 qualifications,...,Medical and care establishment: NHS: Mental health hospital/unit (including secure units),Medical and care establishment: NHS: Other hospital,Medical and care establishment: Other,Medical and care establishment: Other: Care home with nursing,Medical and care establishment: Other: Care home without nursing,Medical and care establishment: Other: Children's home (including secure units),Medical and care establishment: Registered Social Landlord/Housing Association,Other establishments,Total weekly income (£),Median house price
0,E02000053,Barnet 030,J01000055,London,7153.0,262.0,189.0,718.0,86.0,718.0,...,0.0,0.0,0.0,0.0,17.0,0.0,0.0,428.5,810.0,NaN
1,E02000209,Croydon 016,J01000055,London,6136.0,245.0,69.0,255.0,92.0,959.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,810.0,191750
2,E02000054,Barnet 031,J01000055,London,5907.0,175.0,57.0,446.0,64.0,442.0,...,0.0,0.0,5.0,0.0,26.0,0.0,0.0,0.0,950.0,330000
3,E02000055,Barnet 032,J01000055,London,9137.0,345.0,146.0,929.0,79.0,677.0,...,0.0,0.0,0.0,0.0,106.0,0.0,0.0,130.5,870.0,325000
4,E02000056,Barnet 033,J01000055,London,7192.0,83.0,17.0,282.0,45.0,402.0,...,0.0,0.0,0.0,86.0,81.0,0.0,0.0,3.0,1300.0,890000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,E02003999,Carlisle 013,J01000021,Carlisle,7862.0,78.0,15.0,72.0,410.0,1145.0,...,12.0,0.0,0.0,33.0,33.0,0.0,0.0,103.0,690.0,175000
3256,E02005068,Maidstone 001,J01000022,Chatham,5659.0,89.0,12.0,47.0,262.0,913.0,...,0.0,0.0,0.0,63.0,4.0,2.0,0.0,6.0,1020.0,196250
3257,E02004497,Chelmsford 013,J01000023,Chelmsford,6535.0,74.0,7.0,81.0,252.0,876.0,...,0.0,0.0,0.0,39.0,13.0,0.0,0.0,2.0,960.0,242000
3258,E02004493,Chelmsford 009,J01000023,Chelmsford,8147.0,167.0,26.0,157.0,196.0,1107.0,...,0.0,0.0,0.0,128.0,0.0,0.0,0.0,15.5,940.0,202750


In [9]:
c2011_df = c2011_housing_df.copy()
c2011_df = c2011_df.dropna(axis=1, how='all')
c2011_df["Central and South America"] = c2011_df["South America"] + c2011_df["Central America"]
c2011_df["3 or more cars or vans in household"] = c2011_df["3 cars or vans in household"] + c2011_df["4 or more cars or vans in household"]
c2011_df["Married or in a registered civil partnership"] = c2011_df["Married"] + c2011_df["In a registered same-sex civil partnership"]
c2011_df["Age 15 to 19 years"] = c2011_df["Age 15"] + c2011_df["Age 16 to 17"] + c2011_df["Age 18 to 19"]
c2011_df["Aged 5 to 9 years"] = c2011_df["Age 5 to 7"] + c2011_df["Age 8 to 9"]
c2011_df["Aged 85 years and over"] = c2011_df["Age 85 to 89"] + c2011_df["Age 90 and over"]
c2011_df.to_csv(r'.\data\clean\c2011.csv', index=False)

In [10]:
base2001_df = pd.read_csv(r'.\data\raw\geoportal\Middle_Layer_Super_Output_Area_(2001)_to_Middle_Layer_Super_Output_Area_(2011)_to_Local_Authority_District_(2011)_Lookup_in_England_and_Wales.csv')
base2001_df = base2001_df[['MSOA01CD', 'MSOA11CD']]
base2001_df = base2001_df.merge(base2011_df, on='MSOA11CD')
base2001_df

,MSOA01CD,MSOA11CD,MSOA11NM,TCITY15CD,TCITY15NM
0,E02000001,E02000001,City of London 001,J01000055,London
1,E02000101,E02000101,Brent 009,J01000055,London
2,E02000002,E02000002,Barking and Dagenham 001,J01000055,London
3,E02000003,E02000003,Barking and Dagenham 002,J01000055,London
4,E02000102,E02000102,Brent 010,J01000055,London
...,...,...,...,...,...
3271,W02000410,W02000410,Cardiff 044,J01000020,Cardiff
3272,W02000411,W02000411,Cardiff 045,J01000020,Cardiff
3273,W02000412,W02000412,Cardiff 046,J01000020,Cardiff
3274,W02000413,W02000422,Cardiff 048,J01000020,Cardiff


In [11]:
metadata_2001 = pd.read_csv(r'.\data\raw\api\msoa\c2001\metadata.csv')
metadata_2001['clean_mnemonic'] = metadata_2001['mnemonic'].str.extract('(ks.*)')
metadata_2001 = metadata_2001[metadata_2001['geoglevel'].str.contains('msoa')]
metadata_2001_mnemonics = list(set(metadata_2001['clean_mnemonic']))
len(metadata_2001_mnemonics)

32

In [12]:
c2001_dfs = []

for mnemonic in metadata_2001_mnemonics:

    root_dir = r'.\data\raw\api\msoa\c2001'
    mnemonic_tables = glob.glob(mnemonic+'*', root_dir=root_dir)
    mnemonic_dfs = []

    for table in mnemonic_tables:
        df = pd.read_csv(rf"{root_dir}\{table}")
        mnemonic_dfs.append(df)

    mnemonic_df = pd.concat(mnemonic_dfs)

    mnemonic_df = mnemonic_df.drop(columns=['Unnamed: 0', 'GEOGRAPHY_TYPE', 'GEOGRAPHY_TYPECODE', 'DATE'])
    mnemonic_df['OBS_VALUE'] = mnemonic_df['OBS_VALUE'].astype(float)
    mnemonic_df = mnemonic_df[mnemonic_df['OBS_VALUE'] > 0]
    cols = list(mnemonic_df.columns)
    var_col = [col for col in cols if col not in ['GEOGRAPHY_CODE', 'OBS_VALUE']][0]
    mnemonic_df = mnemonic_df.pivot_table(index='GEOGRAPHY_CODE', columns=var_col, values='OBS_VALUE', aggfunc='mean', fill_value=0)
    mnemonic_df = mnemonic_df.reset_index(drop=False)
    c2001_dfs.append(mnemonic_df)
c2001_dfs[0]

C_HHCHUK11_NAME,GEOGRAPHY_CODE,All households,Average household size,Average number of rooms per household,Lowest floor level - Basement or semi-basement,Lowest floor level - Ground level (street level),Lowest floor level - fifth floor or higher,Lowest floor level - first to fourth floor,Occupancy rating of -1 or less,With central heating and sole use of bath/shower and toilet,With central heating; without sole use of bath/shower and toilet,Without central heating or sole use of bath/shower and toilet,Without central heating; with sole use of bath/shower and toilet
0,E02000001,4338.0,1.58,3.33,218.0,435.0,1390.0,2295.0,1486.0,4122.0,17.0,0.0,199.0
1,E02000002,2734.0,2.27,4.57,5.0,2143.0,54.0,532.0,300.0,2253.0,3.0,3.0,475.0
2,E02000003,3784.0,2.43,4.93,13.0,3166.0,4.0,601.0,331.0,3399.0,3.0,6.0,376.0
3,E02000004,2400.0,2.48,5.07,0.0,2196.0,3.0,202.0,142.0,2277.0,4.0,0.0,118.0
4,E02000005,3193.0,2.47,4.70,5.0,2956.0,7.0,225.0,320.0,2812.0,5.0,0.0,376.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7189,W02000409,2451.0,2.40,5.16,5.0,2243.0,0.0,203.0,169.0,2296.0,3.0,3.0,150.0
7190,W02000410,2753.0,2.56,5.50,37.0,2475.0,0.0,240.0,211.0,2320.0,18.0,3.0,413.0
7191,W02000411,2420.0,2.43,5.15,5.0,2130.0,12.0,273.0,202.0,2231.0,7.0,0.0,182.0
7192,W02000412,2489.0,2.34,5.25,48.0,2208.0,3.0,231.0,198.0,2234.0,7.0,4.0,244.0


In [13]:
c2001_census_df = base2001_df.copy()
for df in c2001_dfs:
    c2001_census_df = c2001_census_df.merge(df, how='left', left_on='MSOA01CD', right_on='GEOGRAPHY_CODE', suffixes=('', '_y'))
    c2001_census_df_cols = list(c2001_census_df.columns)
    c2001_census_df_dup_cols = [col for col in c2001_census_df_cols if '_y' in col]
    c2001_census_df_dup_cols.append('GEOGRAPHY_CODE')
    c2001_census_df = c2001_census_df.drop(columns=c2001_census_df_dup_cols)
c2001_census_df = c2001_census_df.dropna(axis=1, how='all')
c2001_census_df

,MSOA01CD,MSOA11CD,MSOA11NM,TCITY15CD,TCITY15NM,All households,Average household size,Average number of rooms per household,Lowest floor level - Basement or semi-basement,Lowest floor level - Ground level (street level),...,Whole house or bungalow - detatched,Whole house or bungalow - semi-detatched,Whole house or bungalow - terraced,1 car or van in household,2 cars or vans in household,3 cars or vans in household,4 or more cars or vans in household,All categories: Car or van availability,No cars or vans in household,sum of all cars or vans in the area
0,E02000001,E02000001,City of London 001,J01000055,London,4338.0,1.58,3.33,218.0,435.0,...,15.0,13.0,74.0,1417.0,184.0,29.0,17.0,4338.0,2691.0,1941.0
1,E02000101,E02000101,Brent 009,J01000055,London,2857.0,2.55,4.63,25.0,1881.0,...,464.0,609.0,264.0,1191.0,518.0,102.0,35.0,2857.0,1011.0,2692.0
2,E02000002,E02000002,Barking and Dagenham 001,J01000055,London,2734.0,2.27,4.57,5.0,2143.0,...,89.0,759.0,930.0,1189.0,376.0,65.0,20.0,2734.0,1084.0,2219.0
3,E02000003,E02000003,Barking and Dagenham 002,J01000055,London,3784.0,2.43,4.93,13.0,3166.0,...,127.0,896.0,1790.0,1841.0,716.0,116.0,33.0,3784.0,1078.0,3760.0
4,E02000102,E02000102,Brent 010,J01000055,London,2727.0,2.81,4.46,24.0,2160.0,...,122.0,531.0,1195.0,1280.0,434.0,78.0,14.0,2727.0,921.0,2440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,W02000410,W02000410,Cardiff 044,J01000020,Cardiff,2753.0,2.56,5.50,37.0,2475.0,...,73.0,180.0,2132.0,1203.0,329.0,43.0,13.0,2753.0,1165.0,2049.0
3272,W02000411,W02000411,Cardiff 045,J01000020,Cardiff,2420.0,2.43,5.15,5.0,2130.0,...,211.0,1126.0,609.0,1055.0,352.0,48.0,17.0,2420.0,948.0,1971.0
3273,W02000412,W02000412,Cardiff 046,J01000020,Cardiff,2489.0,2.34,5.25,48.0,2208.0,...,134.0,578.0,1334.0,1179.0,327.0,43.0,15.0,2489.0,925.0,2028.0
3274,W02000413,W02000422,Cardiff 048,J01000020,Cardiff,2957.0,1.95,4.46,32.0,1837.0,...,79.0,387.0,884.0,1263.0,376.0,46.0,9.0,2957.0,1263.0,2195.0


In [14]:
housing2001_df = pd.read_excel(r'.\data\raw\ons\hpssamedianpricebymsoa.xlsx', sheet_name='1a', header=2)
housing2001_df = housing2001_df[['MSOA code', 'Year ending Dec 2001']]
housing2001_df = housing2001_df.rename(columns={'Year ending Dec 2001': 'Median house price'})
housing2001_df

c2001_housing_df = c2001_census_df.copy()
c2001_housing_df = c2001_housing_df.merge(housing2001_df, how='left', left_on='MSOA01CD', right_on='MSOA code')
c2001_housing_df = c2001_housing_df.drop(columns='MSOA code')
c2001_housing_df

,MSOA01CD,MSOA11CD,MSOA11NM,TCITY15CD,TCITY15NM,All households,Average household size,Average number of rooms per household,Lowest floor level - Basement or semi-basement,Lowest floor level - Ground level (street level),...,Whole house or bungalow - semi-detatched,Whole house or bungalow - terraced,1 car or van in household,2 cars or vans in household,3 cars or vans in household,4 or more cars or vans in household,All categories: Car or van availability,No cars or vans in household,sum of all cars or vans in the area,Median house price
0,E02000001,E02000001,City of London 001,J01000055,London,4338.0,1.58,3.33,218.0,435.0,...,13.0,74.0,1417.0,184.0,29.0,17.0,4338.0,2691.0,1941.0,237500
1,E02000101,E02000101,Brent 009,J01000055,London,2857.0,2.55,4.63,25.0,1881.0,...,609.0,264.0,1191.0,518.0,102.0,35.0,2857.0,1011.0,2692.0,130000
2,E02000002,E02000002,Barking and Dagenham 001,J01000055,London,2734.0,2.27,4.57,5.0,2143.0,...,759.0,930.0,1189.0,376.0,65.0,20.0,2734.0,1084.0,2219.0,95000
3,E02000003,E02000003,Barking and Dagenham 002,J01000055,London,3784.0,2.43,4.93,13.0,3166.0,...,896.0,1790.0,1841.0,716.0,116.0,33.0,3784.0,1078.0,3760.0,115000
4,E02000102,E02000102,Brent 010,J01000055,London,2727.0,2.81,4.46,24.0,2160.0,...,531.0,1195.0,1280.0,434.0,78.0,14.0,2727.0,921.0,2440.0,115500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,W02000410,W02000410,Cardiff 044,J01000020,Cardiff,2753.0,2.56,5.50,37.0,2475.0,...,180.0,2132.0,1203.0,329.0,43.0,13.0,2753.0,1165.0,2049.0,65000
3272,W02000411,W02000411,Cardiff 045,J01000020,Cardiff,2420.0,2.43,5.15,5.0,2130.0,...,1126.0,609.0,1055.0,352.0,48.0,17.0,2420.0,948.0,1971.0,45000
3273,W02000412,W02000412,Cardiff 046,J01000020,Cardiff,2489.0,2.34,5.25,48.0,2208.0,...,578.0,1334.0,1179.0,327.0,43.0,15.0,2489.0,925.0,2028.0,71000
3274,W02000413,W02000422,Cardiff 048,J01000020,Cardiff,2957.0,1.95,4.46,32.0,1837.0,...,387.0,884.0,1263.0,376.0,46.0,9.0,2957.0,1263.0,2195.0,NaN


In [15]:
c2001_df = c2001_housing_df.copy()
c2001_df = c2001_df.dropna(axis=1, how='all')
c2001_df["Provides no unpaid care"] = c2001_df["All usual residents"] - c2001_df["All people who provide unpaid care"]
c2001_df["3 or more cars or vans in household"] = c2001_df["3 cars or vans in household"] + c2001_df["4 or more cars or vans in household"]
c2001_df["Married or in a registered civil partnership"] = c2001_df["Married (first marriage)"] + c2001_df["Re-married"]
c2001_df["Private rented: Other"] = c2001_df["Rented from a housing association/registered social landlord"] + c2001_df["Rented from council(local authority)"]
c2001_df["Age 15 to 19 years"] = c2001_df["Age 15"] + c2001_df["Age 16 to 17"] + c2001_df["Age 18 to 19"]
c2001_df["Aged 5 to 9 years"] = c2001_df["Age 5 to 7"] + c2001_df["Age 8 to 9"]
c2001_df["Aged 85 years and over"] = c2001_df["Age 85 to 89"] + c2001_df["Age 90 and over"]
c2001_df.to_csv(r'.\data\clean\c2001.csv', index=False)

In [16]:
base2021_df = pd.read_csv(r'.\data\raw\geoportal\MSOA_(2011)_to_MSOA_(2021)_to_Local_Authority_District_(2022)_Lookup_for_England_and_Wales_(Version_2).csv')
base2021_df = base2021_df[['MSOA21CD', 'MSOA11CD']]
base2021_df = base2021_df.merge(base2011_df, on='MSOA11CD')
base2021_df

,MSOA21CD,MSOA11CD,MSOA11NM,TCITY15CD,TCITY15NM
0,E02000308,E02000308,Enfield 032,J01000055,London
1,E02000309,E02000309,Enfield 033,J01000055,London
2,E02000311,E02000311,Enfield 035,J01000055,London
3,E02000312,E02000312,Enfield 036,J01000055,London
4,E02000313,E02000313,Greenwich 001,J01000055,London
...,...,...,...,...,...
3255,W02000193,W02000193,Swansea 026,J01000098,Swansea
3256,W02000194,W02000194,Swansea 027,J01000098,Swansea
3257,W02000196,W02000196,Swansea 029,J01000098,Swansea
3258,W02000198,W02000198,Swansea 031,J01000098,Swansea


In [17]:
metadata_2021 = pd.read_csv(r'.\data\raw\api\msoa\c2021\metadata.csv')
metadata_2021['clean_mnemonic'] = metadata_2021['mnemonic'].str.extract('(c2021ts.*)')
metadata_2021 = metadata_2021[metadata_2021['geoglevel'].str.contains('msoa')]
metadata_2021_mnemonics = list(set(metadata_2021['clean_mnemonic']))
len(metadata_2021_mnemonics)

68

In [18]:
c2021_dfs = []

for mnemonic in metadata_2021_mnemonics:
    root_dir = r'.\data\raw\api\msoa\c2021'
    mnemonic_tables = glob.glob(mnemonic+'*', root_dir=root_dir)
    mnemonic_dfs = []

    for table in mnemonic_tables:
        df = pd.read_csv(rf"{root_dir}\{table}")
        mnemonic_dfs.append(df)

    mnemonic_df = pd.concat(mnemonic_dfs)

    mnemonic_df = mnemonic_df.drop(columns=['Unnamed: 0', 'GEOGRAPHY_TYPE', 'GEOGRAPHY_TYPECODE', 'DATE'])
    mnemonic_df['OBS_VALUE'] = mnemonic_df['OBS_VALUE'].astype(float)
    mnemonic_df = mnemonic_df[mnemonic_df['OBS_VALUE'] > 0]
    cols = list(mnemonic_df.columns)
    var_col = [col for col in cols if col not in ['GEOGRAPHY_CODE', 'OBS_VALUE']][0]
    mnemonic_df = mnemonic_df.pivot_table(index='GEOGRAPHY_CODE', columns=var_col, values='OBS_VALUE', aggfunc='mean', fill_value=0)
    mnemonic_df = mnemonic_df.reset_index(drop=False)
    c2021_dfs.append(mnemonic_df)
c2021_dfs[37]

C2021_LPSTAT_12_NAME,GEOGRAPHY_CODE,Divorced or civil partnership dissolved,Divorced or civil partnership dissolved: Divorced,Divorced or civil partnership dissolved: Formerly in a civil partnership now legally dissolved,Married or in a registered civil partnership,Married or in a registered civil partnership: In a registered civil partnership,Married or in a registered civil partnership: In a registered civil partnership: Opposite sex,Married or in a registered civil partnership: In a registered civil partnership: Same sex,Married or in a registered civil partnership: Married,Married or in a registered civil partnership: Married: Opposite sex,Married or in a registered civil partnership: Married: Same sex,Never married and never registered a civil partnership,"Separated, but still legally married or still legally in a civil partnership","Separated, but still legally married or still legally in a civil partnership: Separated, but still in a registered civil partnership","Separated, but still legally married or still legally in a civil partnership: Separated, but still married",Total: All usual residents aged 16 and over,Widowed or surviving civil partnership partner,Widowed or surviving civil partnership partner: Surviving partner from civil partnership,Widowed or surviving civil partnership partner: Widowed
0,E02000001,559.0,537.0,22.0,2435.0,113.0,17.0,96.0,2322.0,2218.0,104.0,4594.0,172.0,6.0,166.0,8003.0,243.0,1.0,242.0
1,E02000002,551.0,551.0,0.0,2445.0,6.0,6.0,0.0,2439.0,2437.0,2.0,2484.0,163.0,2.0,161.0,5984.0,341.0,0.0,341.0
2,E02000003,695.0,694.0,1.0,4101.0,12.0,10.0,2.0,4089.0,4046.0,43.0,3398.0,215.0,2.0,213.0,8846.0,437.0,0.0,437.0
3,E02000004,396.0,395.0,1.0,2381.0,9.0,9.0,0.0,2372.0,2363.0,9.0,2056.0,136.0,0.0,136.0,5316.0,347.0,0.0,347.0
4,E02000005,602.0,602.0,0.0,3703.0,12.0,6.0,6.0,3691.0,3683.0,8.0,3141.0,201.0,3.0,198.0,7987.0,340.0,0.0,340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7259,W02000424,539.0,537.0,2.0,3104.0,6.0,2.0,4.0,3098.0,3093.0,5.0,1539.0,80.0,0.0,80.0,5760.0,498.0,2.0,496.0
7260,W02000425,613.0,611.0,2.0,3212.0,14.0,2.0,12.0,3198.0,3189.0,9.0,2845.0,128.0,0.0,128.0,7180.0,382.0,0.0,382.0
7261,W02000426,826.0,826.0,0.0,3998.0,6.0,2.0,4.0,3992.0,3977.0,15.0,2606.0,121.0,2.0,119.0,8203.0,652.0,0.0,652.0
7262,W02000427,899.0,897.0,2.0,3929.0,20.0,10.0,10.0,3909.0,3882.0,27.0,3329.0,230.0,4.0,226.0,9146.0,759.0,1.0,758.0


In [19]:
c2021_census_df = base2021_df.copy()
for df in c2021_dfs:
    c2021_census_df = c2021_census_df.merge(df, how='left', left_on='MSOA21CD', right_on='GEOGRAPHY_CODE', suffixes=('', '_y'))
    c2021_census_df_cols = list(c2021_census_df.columns)
    c2021_census_df_dup_cols = [col for col in c2021_census_df_cols if '_y' in col]
    c2021_census_df_dup_cols.append('GEOGRAPHY_CODE')
    c2021_census_df = c2021_census_df.drop(columns=c2021_census_df_dup_cols)
c2021_census_df = c2021_census_df.dropna(axis=1, how='all')
c2021_census_df

,MSOA21CD,MSOA11CD,MSOA11NM,TCITY15CD,TCITY15NM,"1. Managers, directors and senior officials",2. Professional occupations,3. Associate professional and technical occupations,4. Administrative and secretarial occupations,5. Skilled trades occupations,...,Arrived 1961 to 1970,Arrived 1971 to 1980,Arrived 1981 to 1990,Arrived 1991 to 2000,Arrived 2001 to 2010,Arrived 2011 to 2013,Arrived 2014 to 2016,Arrived 2017 to 2019,Arrived 2020 to 2021,Arrived before 1951
0,E02000308,E02000308,Enfield 032,J01000055,London,540.0,969.0,626.0,419.0,437.0,...,280.0,302.0,299.0,536.0,958.0,312.0,509.0,590.0,184.0,15.0
1,E02000309,E02000309,Enfield 033,J01000055,London,371.0,655.0,402.0,414.0,552.0,...,345.0,232.0,371.0,1055.0,1888.0,580.0,687.0,730.0,280.0,3.0
2,E02000311,E02000311,Enfield 035,J01000055,London,482.0,846.0,562.0,412.0,606.0,...,472.0,509.0,403.0,754.0,1210.0,362.0,692.0,682.0,328.0,15.0
3,E02000312,E02000312,Enfield 036,J01000055,London,772.0,1376.0,826.0,499.0,658.0,...,404.0,438.0,280.0,524.0,1284.0,512.0,821.0,834.0,248.0,12.0
4,E02000313,E02000313,Greenwich 001,J01000055,London,186.0,399.0,278.0,222.0,235.0,...,56.0,66.0,175.0,442.0,1020.0,225.0,238.0,237.0,64.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,W02000193,W02000193,Swansea 026,J01000098,Swansea,237.0,555.0,328.0,214.0,223.0,...,39.0,52.0,65.0,136.0,454.0,144.0,199.0,341.0,285.0,11.0
3256,W02000194,W02000194,Swansea 027,J01000098,Swansea,339.0,910.0,262.0,217.0,125.0,...,36.0,55.0,51.0,94.0,216.0,43.0,87.0,113.0,185.0,10.0
3257,W02000196,W02000196,Swansea 029,J01000098,Swansea,291.0,651.0,317.0,230.0,200.0,...,26.0,25.0,24.0,49.0,101.0,32.0,54.0,49.0,15.0,5.0
3258,W02000198,W02000198,Swansea 031,J01000098,Swansea,549.0,991.0,441.0,250.0,213.0,...,43.0,29.0,35.0,53.0,99.0,35.0,59.0,56.0,17.0,8.0


In [20]:
list(c2021_census_df.columns)

['MSOA21CD',
 'MSOA11CD',
 'MSOA11NM',
 'TCITY15CD',
 'TCITY15NM',
 '1. Managers, directors and senior officials',
 '2. Professional occupations',
 '3. Associate professional and technical occupations',
 '4. Administrative and secretarial occupations',
 '5. Skilled trades occupations',
 '6. Caring, leisure and other service occupations',
 '7. Sales and customer service occupations',
 '8. Process, plant and machine operatives',
 '9. Elementary occupations',
 'Total: All usual residents aged 16 years and over in employment the week before the census',
 'Apprenticeship',
 'Level 1 and entry level qualifications',
 'Level 2 qualifications',
 'Level 3 qualifications',
 'Level 4 qualifications or above',
 'No qualifications',
 'Other qualifications',
 'Total: All usual residents aged 16 years and over',
 '10km to less than 20km',
 '20km to less than 30km',
 '2km to less than 5km',
 '30km to less than 40km',
 '40km to less than 60km',
 '5km to less than 10km',
 '60km and over',
 'Less than 2k

In [21]:
income2021_df = pd.read_excel(r'.\data\raw\ons\saiefy1920finalqaddownload280923.xlsx', sheet_name='Total annual income', header=4)
income2021_df = income2021_df[['MSOA code', 'Total annual income (£)']]
income2021_df = income2021_df.dropna(axis=0, how='any')
income2021_df

c2021_income_df = c2021_census_df.copy()
c2021_income_df = c2021_income_df.merge(income2021_df, how='left', left_on='MSOA21CD', right_on='MSOA code')
c2021_income_df = c2021_income_df.drop(columns='MSOA code')
c2021_income_df

,MSOA21CD,MSOA11CD,MSOA11NM,TCITY15CD,TCITY15NM,"1. Managers, directors and senior officials",2. Professional occupations,3. Associate professional and technical occupations,4. Administrative and secretarial occupations,5. Skilled trades occupations,...,Arrived 1971 to 1980,Arrived 1981 to 1990,Arrived 1991 to 2000,Arrived 2001 to 2010,Arrived 2011 to 2013,Arrived 2014 to 2016,Arrived 2017 to 2019,Arrived 2020 to 2021,Arrived before 1951,Total annual income (£)
0,E02000308,E02000308,Enfield 032,J01000055,London,540.0,969.0,626.0,419.0,437.0,...,302.0,299.0,536.0,958.0,312.0,509.0,590.0,184.0,15.0,57000.0
1,E02000309,E02000309,Enfield 033,J01000055,London,371.0,655.0,402.0,414.0,552.0,...,232.0,371.0,1055.0,1888.0,580.0,687.0,730.0,280.0,3.0,42500.0
2,E02000311,E02000311,Enfield 035,J01000055,London,482.0,846.0,562.0,412.0,606.0,...,509.0,403.0,754.0,1210.0,362.0,692.0,682.0,328.0,15.0,52800.0
3,E02000312,E02000312,Enfield 036,J01000055,London,772.0,1376.0,826.0,499.0,658.0,...,438.0,280.0,524.0,1284.0,512.0,821.0,834.0,248.0,12.0,59400.0
4,E02000313,E02000313,Greenwich 001,J01000055,London,186.0,399.0,278.0,222.0,235.0,...,66.0,175.0,442.0,1020.0,225.0,238.0,237.0,64.0,3.0,41400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,W02000193,W02000193,Swansea 026,J01000098,Swansea,237.0,555.0,328.0,214.0,223.0,...,52.0,65.0,136.0,454.0,144.0,199.0,341.0,285.0,11.0,31900.0
3256,W02000194,W02000194,Swansea 027,J01000098,Swansea,339.0,910.0,262.0,217.0,125.0,...,55.0,51.0,94.0,216.0,43.0,87.0,113.0,185.0,10.0,47600.0
3257,W02000196,W02000196,Swansea 029,J01000098,Swansea,291.0,651.0,317.0,230.0,200.0,...,25.0,24.0,49.0,101.0,32.0,54.0,49.0,15.0,5.0,39200.0
3258,W02000198,W02000198,Swansea 031,J01000098,Swansea,549.0,991.0,441.0,250.0,213.0,...,29.0,35.0,53.0,99.0,35.0,59.0,56.0,17.0,8.0,47800.0


In [22]:
housing2021_df = pd.read_excel(r'.\data\raw\ons\hpssamedianpricebymsoa.xlsx', sheet_name='1a', header=2)
housing2021_df = housing2021_df[['MSOA code', 'Year ending Dec 2021']]
housing2021_df = housing2021_df.rename(columns={'Year ending Dec 2021': 'Median house price'})
housing2021_df

c2021_housing_df = c2021_income_df.copy()
c2021_housing_df = c2021_housing_df.merge(housing2021_df, how='left', left_on='MSOA21CD', right_on='MSOA code')
c2021_housing_df = c2021_housing_df.drop(columns='MSOA code')
c2021_housing_df

,MSOA21CD,MSOA11CD,MSOA11NM,TCITY15CD,TCITY15NM,"1. Managers, directors and senior officials",2. Professional occupations,3. Associate professional and technical occupations,4. Administrative and secretarial occupations,5. Skilled trades occupations,...,Arrived 1981 to 1990,Arrived 1991 to 2000,Arrived 2001 to 2010,Arrived 2011 to 2013,Arrived 2014 to 2016,Arrived 2017 to 2019,Arrived 2020 to 2021,Arrived before 1951,Total annual income (£),Median house price
0,E02000308,E02000308,Enfield 032,J01000055,London,540.0,969.0,626.0,419.0,437.0,...,299.0,536.0,958.0,312.0,509.0,590.0,184.0,15.0,57000.0,468500.0
1,E02000309,E02000309,Enfield 033,J01000055,London,371.0,655.0,402.0,414.0,552.0,...,371.0,1055.0,1888.0,580.0,687.0,730.0,280.0,3.0,42500.0,382500.0
2,E02000311,E02000311,Enfield 035,J01000055,London,482.0,846.0,562.0,412.0,606.0,...,403.0,754.0,1210.0,362.0,692.0,682.0,328.0,15.0,52800.0,469975.0
3,E02000312,E02000312,Enfield 036,J01000055,London,772.0,1376.0,826.0,499.0,658.0,...,280.0,524.0,1284.0,512.0,821.0,834.0,248.0,12.0,59400.0,577500.0
4,E02000313,E02000313,Greenwich 001,J01000055,London,186.0,399.0,278.0,222.0,235.0,...,175.0,442.0,1020.0,225.0,238.0,237.0,64.0,3.0,41400.0,277500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,W02000193,W02000193,Swansea 026,J01000098,Swansea,237.0,555.0,328.0,214.0,223.0,...,65.0,136.0,454.0,144.0,199.0,341.0,285.0,11.0,31900.0,157750.0
3256,W02000194,W02000194,Swansea 027,J01000098,Swansea,339.0,910.0,262.0,217.0,125.0,...,51.0,94.0,216.0,43.0,87.0,113.0,185.0,10.0,47600.0,324000.0
3257,W02000196,W02000196,Swansea 029,J01000098,Swansea,291.0,651.0,317.0,230.0,200.0,...,24.0,49.0,101.0,32.0,54.0,49.0,15.0,5.0,39200.0,266000.0
3258,W02000198,W02000198,Swansea 031,J01000098,Swansea,549.0,991.0,441.0,250.0,213.0,...,35.0,53.0,99.0,35.0,59.0,56.0,17.0,8.0,47800.0,362500.0


In [23]:
c2021_df = c2021_housing_df.copy()
c2021_df = c2021_df.dropna(axis=1, how='all')
c2021_df["Average number of bedrooms per household"] = (c2021_df["1 bedroom"] + c2021_df["2 bedrooms"] * 2 + c2021_df["3 bedrooms"] * 3 + c2021_df["4 or more bedrooms"] * 5) / c2021_df["Number of households"]
c2021_df["Other combination of skills in Welsh"] = c2021_df["Can read and write but cannot speak Welsh"] + c2021_df["Can read but cannot speak or write Welsh"] + c2021_df["Can speak and other combinations of skills in Welsh"] + c2021_df["Can speak, read and write Welsh"] + c2021_df["Can understand spoken Welsh only"] + c2021_df["Can write but cannot speak or read Welsh"]
c2021_df["Economically active: Self-employed"] = c2021_df["Economically active and a full-time student:In employment:Self-employed with employees"] + c2021_df["Economically active and a full-time student:In employment:Self-employed without employees"]
c2021_df["Economically active: Full-time student"] = c2021_df["Economically active and a full-time student"] - c2021_df["Economically active (excluding full-time students)"]
c2021_df["Medical and care establishment: Local Authority: Care home or other home"] = c2021_df["Medical and care establishment: Local Authority: Care home with nursing"] + c2021_df["Medical and care establishment: Local Authority: Care home without nursing"] + c2021_df["Medical and care establishment: Local Authority: Other home"]
c2021_df["Age 30 to 44"] = c2021_df["Aged 30 to 34 years"] + c2021_df["Aged 35 to 39 years"] + c2021_df["Aged 40 to 44 years"]
c2021_df["Age 45 to 59"] = c2021_df["Aged 45 to 49 years"] + c2021_df["Aged 50 to 54 years"] + c2021_df["Aged 55 to 59 years"]
c2021_df["Age 65 to 74"] = c2021_df["Aged 65 to 69 years"] + c2021_df["Aged 70 to 74 years"]
c2021_df["Age 75 to 84"] = c2021_df["Aged 75 to 79 years"] + c2021_df["Aged 80 to 84 years"]

c2021_df.to_csv(r'.\data\clean\c2021.csv', index=False)
c2021_df

,MSOA21CD,MSOA11CD,MSOA11NM,TCITY15CD,TCITY15NM,"1. Managers, directors and senior officials",2. Professional occupations,3. Associate professional and technical occupations,4. Administrative and secretarial occupations,5. Skilled trades occupations,...,Median house price,Average number of bedrooms per household,Other combination of skills in Welsh,Economically active: Self-employed,Economically active: Full-time student,Medical and care establishment: Local Authority: Care home or other home,Age 30 to 44,Age 45 to 59,Age 65 to 74,Age 75 to 84
0,E02000308,E02000308,Enfield 032,J01000055,London,540.0,969.0,626.0,419.0,437.0,...,468500.0,2.858648,NaN,34.0,-4267.0,0.0,2210.0,1689.0,562.0,364.0
1,E02000309,E02000309,Enfield 033,J01000055,London,371.0,655.0,402.0,414.0,552.0,...,382500.0,2.581244,NaN,25.0,-4661.0,0.0,2583.0,2251.0,602.0,301.0
2,E02000311,E02000311,Enfield 035,J01000055,London,482.0,846.0,562.0,412.0,606.0,...,469975.0,2.815406,NaN,31.0,-4885.0,0.0,2390.0,2096.0,790.0,415.0
3,E02000312,E02000312,Enfield 036,J01000055,London,772.0,1376.0,826.0,499.0,658.0,...,577500.0,2.600222,NaN,39.0,-5901.0,0.0,3207.0,2175.0,822.0,457.0
4,E02000313,E02000313,Greenwich 001,J01000055,London,186.0,399.0,278.0,222.0,235.0,...,277500.0,2.348998,NaN,17.0,-2793.0,0.0,1349.0,1488.0,397.0,147.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,W02000193,W02000193,Swansea 026,J01000098,Swansea,237.0,555.0,328.0,214.0,223.0,...,157750.0,2.897117,1206.0,42.0,-1364.0,0.0,1082.0,1021.0,526.0,354.0
3256,W02000194,W02000194,Swansea 027,J01000098,Swansea,339.0,910.0,262.0,217.0,125.0,...,324000.0,3.584056,1133.0,5.0,-2059.0,0.0,899.0,1017.0,773.0,566.0
3257,W02000196,W02000196,Swansea 029,J01000098,Swansea,291.0,651.0,317.0,230.0,200.0,...,266000.0,3.030243,956.0,1.0,-2300.0,0.0,977.0,1231.0,927.0,698.0
3258,W02000198,W02000198,Swansea 031,J01000098,Swansea,549.0,991.0,441.0,250.0,213.0,...,362500.0,3.428656,1210.0,5.0,-2853.0,0.0,955.0,1554.0,1187.0,787.0


In [24]:
list(c2001_df.columns)

['MSOA01CD',
 'MSOA11CD',
 'MSOA11NM',
 'TCITY15CD',
 'TCITY15NM',
 'All households',
 'Average household size',
 'Average number of rooms per household',
 'Lowest floor level - Basement or semi-basement',
 'Lowest floor level - Ground level (street level)',
 'Lowest floor level - fifth floor or higher',
 'Lowest floor level - first to fourth floor',
 'Occupancy rating of -1 or less',
 'With central heating and sole use of bath/shower and toilet',
 'With central heating; without sole use of bath/shower and toilet',
 'Without central heating or sole use of bath/shower and toilet',
 'Without central heating; with sole use of bath/shower and toilet',
 'A Agriculture, hunting, forestry',
 'All categories: Industry',
 'B Fishing',
 'C Mining and quarrying',
 'D Manufacturing',
 'E Electricity, gas and water supply',
 'F Constructiion',
 'G Wholesale and retail trade, repair of motor vehicles',
 'H Hotels and restaurants',
 'I Transport storage and communications',
 'J Financial Intermediati

In [26]:
normalizer_df = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '1nLWtw-6VwOgKvADQEhwIa0s9zst_-_aY5CVGEoCoKpU' +
                   '/export?gid=508974035&format=csv')
final_cols = list(normalizer_df['var_final'])
var_types = list(normalizer_df['var_type'])

normcols_2001 = list(normalizer_df['var_2001'])
multcols_2001 = list(normalizer_df['multiplier_2001'])

fin2001_df = pd.DataFrame()

for final_col, var_type, normcol, multcol in zip(final_cols, var_types, normcols_2001, multcols_2001):
    if var_type == 'disc':
        fin2001_df[final_col] = c2001_df[normcol]
    else:
        if not math.isnan(multcol):
            fin2001_df[final_col] = c2001_df[normcol] * multcol
        else:
            fin2001_df[final_col] = 0

fin2001_df['Year'] = 2001
fin2001_df

,MSOA Code,MSOA Name,City Code,City Name,Residency: Average number of bedrooms per household,Sex: Females,Sex: Males,Residency: Lives in a communal establishment,Occupancy rating (rooms) of -1 or less,Passport: Africa,...,Age: 25 to 29,Age: 30 to 44,Age: 45 to 59,Age: 0 to 4,Age: 5 to 9,Age: 60 to 64,Age: 65 to 74,Age: 75 to 84,Age: 85 years and over,Year
0,E02000001,City of London 001,J01000055,London,0,3353.0,3832.0,324.0,743.0,0,...,940.0,1977.0,1622.0,250.0,207.0,306.0,514.0,322.0,126.0,2001
1,E02000101,Brent 009,J01000055,London,0,3955.0,3476.0,143.0,330.0,0,...,590.0,1671.0,1261.0,449.0,487.0,305.0,530.0,392.0,176.0,2001
2,E02000002,Barking and Dagenham 001,J01000055,London,0,3366.0,2871.0,20.0,150.0,0,...,409.0,1277.0,894.0,489.0,430.0,216.0,665.0,528.0,161.0,2001
3,E02000003,Barking and Dagenham 002,J01000055,London,0,4788.0,4402.0,6.0,165.5,0,...,656.0,2227.0,1575.0,614.0,584.0,377.0,730.0,529.0,162.0,2001
4,E02000102,Brent 010,J01000055,London,0,3924.0,3729.0,0.0,379.5,0,...,692.0,1993.0,1045.0,575.0,527.0,330.0,443.0,208.0,70.0,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,W02000410,Cardiff 044,J01000020,Cardiff,0,3566.0,3486.0,3.0,105.5,0,...,609.0,1669.0,948.0,602.0,557.0,262.0,450.0,294.0,73.0,2001
3272,W02000411,Cardiff 045,J01000020,Cardiff,0,3045.0,2861.0,37.0,101.0,0,...,362.0,1339.0,1013.0,430.0,486.0,247.0,388.0,297.0,67.0,2001
3273,W02000412,Cardiff 046,J01000020,Cardiff,0,3011.0,2842.0,29.0,99.0,0,...,510.0,1429.0,906.0,439.0,440.0,234.0,427.0,265.0,82.0,2001
3274,W02000422,Cardiff 048,J01000020,Cardiff,0,2836.0,3113.0,180.0,186.5,0,...,809.0,1415.0,731.0,384.0,355.0,245.0,380.0,261.0,93.0,2001


In [27]:
normcols_2011 = list(normalizer_df['var_2011'])
multcols_2011 = list(normalizer_df['multiplier_2011'])

fin2011_df = pd.DataFrame()

for final_col, var_type, normcol, multcol in zip(final_cols, var_types, normcols_2011, multcols_2011):
    if var_type == 'disc':
        fin2011_df[final_col] = c2011_df[normcol]
    else:
        if not math.isnan(multcol):
            fin2011_df[final_col] = c2011_df[normcol] * multcol
        else:
            fin2011_df[final_col] = 0

fin2011_df['Year'] = 2011
fin2011_df

,MSOA Code,MSOA Name,City Code,City Name,Residency: Average number of bedrooms per household,Sex: Females,Sex: Males,Residency: Lives in a communal establishment,Occupancy rating (rooms) of -1 or less,Passport: Africa,...,Age: 25 to 29,Age: 30 to 44,Age: 45 to 59,Age: 0 to 4,Age: 5 to 9,Age: 60 to 64,Age: 65 to 74,Age: 75 to 84,Age: 85 years and over,Year
0,E02000053,Barnet 030,J01000055,London,2.2,4277.0,4420.0,447.0,616.5,457.0,...,1140.0,2203.0,1271.0,670.0,427.0,304.0,398.0,235.0,74.0,2011
1,E02000209,Croydon 016,J01000055,London,2.5,4310.0,4104.0,3.0,296.5,302.0,...,616.0,2067.0,1495.0,797.0,693.0,259.0,348.0,227.0,60.0,2011
2,E02000054,Barnet 031,J01000055,London,2.6,3654.0,3552.0,31.0,341.5,333.0,...,821.0,1797.0,1218.0,504.0,375.0,321.0,412.0,262.0,94.0,2011
3,E02000055,Barnet 032,J01000055,London,2.6,5616.0,5514.0,237.0,553.0,421.0,...,1364.0,2488.0,1517.0,798.0,608.0,423.0,628.0,530.0,317.0,2011
4,E02000056,Barnet 033,J01000055,London,3.4,4893.0,4319.0,170.0,133.5,234.0,...,641.0,2039.0,1623.0,669.0,648.0,559.0,709.0,456.0,366.0,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,E02003999,Carlisle 013,J01000021,Carlisle,3.0,4795.0,4585.0,181.0,41.5,8.0,...,387.0,1685.0,2111.0,413.0,476.0,775.0,1100.0,683.0,274.0,2011
3256,E02005068,Maidstone 001,J01000022,Chatham,3.2,3552.0,3490.0,75.0,44.5,35.0,...,396.0,1536.0,1627.0,401.0,438.0,480.0,538.0,256.0,104.0,2011
3257,E02004497,Chelmsford 013,J01000023,Chelmsford,3.0,4007.0,3841.0,69.0,61.5,46.0,...,344.0,1433.0,1648.0,354.0,435.0,577.0,937.0,623.0,262.0,2011
3258,E02004493,Chelmsford 009,J01000023,Chelmsford,2.7,5105.0,5022.0,144.0,156.0,138.0,...,963.0,2380.0,1791.0,760.0,600.0,458.0,590.0,475.0,241.0,2011


In [28]:
normcols_2021 = list(normalizer_df['var_2021'])
multcols_2021 = list(normalizer_df['multiplier_2021'])

fin2021_df = pd.DataFrame()

for final_col, var_type, normcol, multcol in zip(final_cols, var_types, normcols_2021, multcols_2021):
    if var_type == 'disc':
        fin2021_df[final_col] = c2021_df[normcol]
    else:
        if not math.isnan(multcol):
            fin2021_df[final_col] = c2021_df[normcol] * multcol
        else:
            fin2021_df[final_col] = 0

fin2021_df['Year'] = 2021
fin2021_df

,MSOA Code,MSOA Name,City Code,City Name,Residency: Average number of bedrooms per household,Sex: Females,Sex: Males,Residency: Lives in a communal establishment,Occupancy rating (rooms) of -1 or less,Passport: Africa,...,Age: 25 to 29,Age: 30 to 44,Age: 45 to 59,Age: 0 to 4,Age: 5 to 9,Age: 60 to 64,Age: 65 to 74,Age: 75 to 84,Age: 85 years and over,Year
0,E02000308,Enfield 032,J01000055,London,2.858648,4433.0,4326.0,16.0,129.0,622.0,...,711.0,2210.0,1689.0,532.0,494.0,429.0,562.0,364.0,138.0,2021
1,E02000309,Enfield 033,J01000055,London,2.581244,6050.0,5744.0,51.0,270.0,1876.0,...,879.0,2583.0,2251.0,875.0,1045.0,513.0,602.0,301.0,186.0,2021
2,E02000311,Enfield 035,J01000055,London,2.815406,5378.0,5166.0,9.0,183.0,743.0,...,866.0,2390.0,2096.0,647.0,598.0,602.0,790.0,415.0,182.0,2021
3,E02000312,Enfield 036,J01000055,London,2.600222,5754.0,5549.0,43.0,193.0,679.0,...,896.0,3207.0,2175.0,780.0,606.0,562.0,822.0,457.0,198.0,2021
4,E02000313,Greenwich 001,J01000055,London,2.348998,3337.0,3152.0,10.0,111.0,1562.0,...,414.0,1349.0,1488.0,434.0,475.0,333.0,397.0,147.0,40.0,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,W02000193,Swansea 026,J01000098,Swansea,2.897117,4046.0,4771.0,181.0,44.0,247.0,...,605.0,1082.0,1021.0,205.0,205.0,299.0,526.0,354.0,186.0,2021
3256,W02000194,Swansea 027,J01000098,Swansea,3.584056,3442.0,3095.0,1082.0,1.0,130.0,...,264.0,899.0,1017.0,221.0,287.0,387.0,773.0,566.0,308.0,2021
3257,W02000196,Swansea 029,J01000098,Swansea,3.030243,3303.0,2975.0,72.0,9.0,48.0,...,251.0,977.0,1231.0,260.0,313.0,438.0,927.0,698.0,287.0,2021
3258,W02000198,Swansea 031,J01000098,Swansea,3.428656,3782.0,3474.0,120.0,2.0,67.0,...,235.0,955.0,1554.0,243.0,305.0,551.0,1187.0,787.0,417.0,2021


In [29]:
fin_df = pd.concat([fin2001_df, fin2011_df, fin2021_df])
id_cols = ["MSOA Code", "MSOA Name", "City Code", "City Name", "Year"]
val_cols = [col for col in list(fin_df.columns) if col not in id_cols]
val_cols = sorted(val_cols)
fin_cols = id_cols + val_cols
fin_df = fin_df[fin_cols]
fin_df = fin_df.groupby(id_cols).sum().reset_index()
fin_df.to_csv(r".\data\clean\normalized_all.csv", index=False)
fin_df

,MSOA Code,MSOA Name,City Code,City Name,Year,Age: 0 to 4,Age: 10 to 14,Age: 15 to 19,Age: 20 to 24,Age: 25 to 29,...,Unpaid Care: Provides no unpaid care,Vehicles: 1 car or van in household,Vehicles: 2 cars or vans in household,Vehicles: 3 or more cars or vans in household,Vehicles: No cars or vans in household,Welsh: Can speak Welsh,Welsh: Can speak and read but cannot write Welsh,Welsh: Can speak but cannot read or write Welsh,Welsh: No skills in Welsh,Welsh: Other combination of skills in Welsh
0,E02000001,City of London 001,J01000055,London,2001,250.0,186.0,206.0,529.0,940.0,...,6587.0,1417.0,184.0,46.0,2691.0,0.0,0.0,0.0,0.0,0.0
1,E02000001,City of London 001,J01000055,London,2011,236.0,169.0,197.0,545.0,956.0,...,6799.0,1100.0,173.0,69.0,3043.0,0.0,0.0,0.0,0.0,0.0
2,E02000001,City of London 001,J01000055,London,2021,213.0,174.0,216.0,965.0,1213.0,...,7876.0,954.0,123.0,43.0,3793.0,0.0,0.0,0.0,0.0,0.0
3,E02000002,Barking and Dagenham 001,J01000055,London,2001,489.0,423.0,382.0,363.0,409.0,...,5624.0,1189.0,376.0,85.0,1084.0,0.0,0.0,0.0,0.0,0.0
4,E02000002,Barking and Dagenham 001,J01000055,London,2011,628.0,446.0,434.0,424.0,510.0,...,6157.0,1186.0,424.0,83.0,1020.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9774,W02000422,Cardiff 048,J01000020,Cardiff,2011,294.0,66.0,110.0,940.0,1725.0,...,5747.0,2299.0,637.0,76.0,642.0,632.0,57.0,62.0,4857.0,107.0
9775,W02000422,Cardiff 048,J01000020,Cardiff,2021,365.0,133.0,124.0,906.0,1665.0,...,6684.0,2462.0,630.0,60.0,1063.0,746.0,31.0,68.0,6012.0,1050.0
9776,W02000423,Cardiff 049,J01000020,Cardiff,2001,384.0,334.0,330.0,612.0,809.0,...,5504.0,1263.0,376.0,55.0,1263.0,0.0,39.0,95.0,4920.0,110.0
9777,W02000423,Cardiff 049,J01000020,Cardiff,2011,543.0,313.0,344.0,967.0,1543.0,...,7662.0,2006.0,515.0,69.0,1542.0,687.0,55.0,99.0,6792.0,186.0
